# Limpieza y Exploración Inicial  
## Tabla `player_data`

En este notebook trabajaremos con la tabla **player_data.csv**, que contiene información biográfica, antropométrica y académica de jugadores de la NBA.  

### Objetivos
1. Revisar la **estructura y calidad** de los datos.  
2. Detectar y manejar **valores faltantes** en variables clave (`college`, `birth_date`, `weight`, `height`, `position`).  
3. Convertir variables de formato textual a numérico/fecha (`height`, `weight`, `birth_date`).  
4. Estandarizar categorías de posición y universidades.  
5. Exportar un dataset limpio como **`player_data_ready.csv`** para integrarlo con los demás (`common`, `combine`, `game`).  


In [1]:
# Librerías base para análisis
import pandas as pd
import numpy as np

# Ruta al archivo crudo en data_raw
path_player = "../data_raw/player_data.csv"

# Cargar dataset
df_player = pd.read_csv(path_player)

# Vista inicial
print("Shape:", df_player.shape)
print("\nColumnas disponibles:", df_player.columns.tolist())
display(df_player.head())



Shape: (4550, 8)

Columnas disponibles: ['name', 'year_start', 'year_end', 'position', 'height', 'weight', 'birth_date', 'college']


,name,year_start,year_end,position,height,weight,birth_date,college
0,Alaa Abdelnaby,1991,1995,F-C,6-10,240.0,"June 24, 1968",Duke University
1,Zaid Abdul-Aziz,1969,1978,C-F,6-9,235.0,"April 7, 1946",Iowa State University
2,Kareem Abdul-Jabbar,1970,1989,C,7-2,225.0,"April 16, 1947","University of California, Los Angeles"
3,Mahmoud Abdul-Rauf,1991,2001,G,6-1,162.0,"March 9, 1969",Louisiana State University
4,Tariq Abdul-Wahad,1998,2003,F,6-6,223.0,"November 3, 1974",San Jose State University


In [2]:
# Información general
df_player.info()

# Nulos por columna
print("\nNulos por columna:")
print(df_player.isna().sum())

# Resumen estadístico de variables numéricas y categóricas
display(df_player.describe(include="all"))



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4550 entries, 0 to 4549
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   name        4550 non-null   object 
 1   year_start  4550 non-null   int64  
 2   year_end    4550 non-null   int64  
 3   position    4549 non-null   object 
 4   height      4549 non-null   object 
 5   weight      4544 non-null   float64
 6   birth_date  4519 non-null   object 
 7   college     4248 non-null   object 
dtypes: float64(1), int64(2), object(5)
memory usage: 284.5+ KB

Nulos por columna:
name            0
year_start      0
year_end        0
position        1
height          1
weight          6
birth_date     31
college       302
dtype: int64


,name,year_start,year_end,position,height,weight,birth_date,college
count,4550,4550.000000,4550.000000,4549,4549,4544.000000,4519,4248
unique,4500,NaN,NaN,7,28,NaN,4161,473
top,George Johnson,NaN,NaN,G,6-7,NaN,"December 31, 1923",University of Kentucky
freq,3,NaN,NaN,1574,473,NaN,3,99
mean,NaN,1985.076264,1989.272527,NaN,NaN,208.908011,NaN,NaN
std,NaN,20.974188,21.874761,NaN,NaN,26.268662,NaN,NaN
min,NaN,1947.000000,1947.000000,NaN,NaN,114.000000,NaN,NaN
25%,NaN,1969.000000,1973.000000,NaN,NaN,190.000000,NaN,NaN
50%,NaN,1986.000000,1992.000000,NaN,NaN,210.000000,NaN,NaN
75%,NaN,2003.000000,2009.000000,NaN,NaN,225.000000,NaN,NaN


### Normalización de nombres

Para asegurar la unión entre `player_data`, `common` y `combine`, se crea la columna `player_name_clean`, aplicando limpieza de caracteres especiales, espacios y mayúsculas/minúsculas.


In [3]:
import re

def clean_name(name):
    if pd.isna(name):
        return None
    name = re.sub(r"[^a-zA-Z\s]", "", name)   # solo letras y espacios
    return name.title().strip()

df_player["player_name_clean"] = df_player["name"].apply(clean_name)

df_player[["name", "player_name_clean"]].head(10)



,name,player_name_clean
0,Alaa Abdelnaby,Alaa Abdelnaby
1,Zaid Abdul-Aziz,Zaid Abdulaziz
2,Kareem Abdul-Jabbar,Kareem Abduljabbar
3,Mahmoud Abdul-Rauf,Mahmoud Abdulrauf
4,Tariq Abdul-Wahad,Tariq Abdulwahad
5,Shareef Abdur-Rahim,Shareef Abdurrahim
6,Tom Abernethy,Tom Abernethy
7,Forest Able,Forest Able
8,John Abramovic,John Abramovic
9,Alex Abrines,Alex Abrines


In [4]:
from datetime import datetime
import numpy as np

# ======================================================
# 1. Altura y peso
# ======================================================
def height_to_m(h):
    try:
        feet, inches = h.split("-")
        total_inches = int(feet) * 12 + int(inches)
        return round(total_inches * 0.0254, 1)
    except:
        return np.nan

df_player["height_m"] = df_player["height"].apply(height_to_m)
df_player["weight_kg"] = (df_player["weight"] * 0.453592).round(1)

# ======================================================
# 2. Fechas y edades
# ======================================================
df_player["birth_date"] = pd.to_datetime(df_player["birth_date"], errors="coerce")

df_player["age_at_debut"] = (
    df_player["year_start"] - df_player["birth_date"].dt.year
).round().astype("Int64")

df_player["age_at_last_season"] = (
    df_player["year_end"] - df_player["birth_date"].dt.year
).round().astype("Int64")

# ======================================================
# 3. Posiciones
# ======================================================
df_player["position"] = df_player["position"].str.replace("-", "/")

# columna con todas las posiciones (limpia)
df_player["position_all"] = df_player["position"]

# columna con posición principal
df_player["position_main"] = df_player["position"].apply(
    lambda x: x.split("/")[0] if pd.notnull(x) else np.nan
)

# ======================================================
# 4. Universidad
# ======================================================
df_player["college"] = df_player["college"].fillna("Unknown")

# ======================================================
# Vista de control
# ======================================================
df_player[[
    "name","height","height_m","weight","weight_kg",
    "birth_date","age_at_debut","age_at_last_season",
    "position_all","position_main","college"
]].head(10)


,name,height,height_m,weight,weight_kg,birth_date,age_at_debut,age_at_last_season,position_all,position_main,college
0,Alaa Abdelnaby,6-10,2.1,240.0,108.9,1968-06-24,23,27,F/C,F,Duke University
1,Zaid Abdul-Aziz,6-9,2.1,235.0,106.6,1946-04-07,23,32,C/F,C,Iowa State University
2,Kareem Abdul-Jabbar,7-2,2.2,225.0,102.1,1947-04-16,23,42,C,C,"University of California, Los Angeles"
3,Mahmoud Abdul-Rauf,6-1,1.9,162.0,73.5,1969-03-09,22,32,G,G,Louisiana State University
4,Tariq Abdul-Wahad,6-6,2.0,223.0,101.2,1974-11-03,24,29,F,F,San Jose State University
5,Shareef Abdur-Rahim,6-9,2.1,225.0,102.1,1976-12-11,21,32,F,F,University of California
6,Tom Abernethy,6-7,2.0,220.0,99.8,1954-05-06,23,27,F,F,Indiana University
7,Forest Able,6-3,1.9,180.0,81.6,1932-07-27,25,25,G,G,Western Kentucky University
8,John Abramovic,6-3,1.9,195.0,88.5,1919-02-09,28,29,F,F,Salem International University
9,Alex Abrines,6-6,2.0,190.0,86.2,1993-08-01,24,25,G/F,G,Unknown


## Transformaciones aplicadas en `player_data`

1. **Altura y peso**
   - `height` (pies-pulgadas) convertido a `height_m` en metros (1 decimal).
   - `weight` (libras) convertido a `weight_kg` en kilogramos (1 decimal).

2. **Fechas y edades**
   - `birth_date` convertido a tipo `datetime`.
   - `age_at_debut` → calculado con `year_start - birth_year`.
   - `age_at_last_season` → calculado con `year_end - birth_year`.

3. **Posiciones**
   - `position` → guiones reemplazados por `/`.
   - `position_all` → conserva todas las posiciones del jugador.
   - `position_main` → toma la primera posición como principal.

4. **Universidad**
   - Se completaron los valores faltantes de `college` con `"Unknown"`.


In [5]:
# Eliminar columnas redundantes
cols_drop = ["height", "weight", "position"]
df_player = df_player.drop(columns=cols_drop)

# Confirmar que se borraron
print("Columnas actuales:")
print(df_player.columns.tolist())


Columnas actuales:
['name', 'year_start', 'year_end', 'birth_date', 'college', 'player_name_clean', 'height_m', 'weight_kg', 'age_at_debut', 'age_at_last_season', 'position_all', 'position_main']


## Eliminación de columnas redundantes

- Se eliminaron las columnas originales `height`, `weight` y `position` porque ya fueron normalizadas:
  - `height` → reemplazada por `height_m` (metros, 1 decimal).
  - `weight` → reemplazada por `weight_kg` (kg, 1 decimal).
  - `position` → reemplazada por `position_all` y `position_main`.

### Columnas actuales en `player_data`
- **Identificación**: `name`, `player_name_clean`.
- **Carrera**: `year_start`, `year_end`.
- **Datos biográficos**: `birth_date`, `college`.
- **Antropometría**: `height_m`, `weight_kg`.
- **Edades**: `age_at_debut`, `age_at_last_season`.
- **Posiciones**: `position_all`, `position_main`.

In [7]:
# Cargar datasets limpios
df_common  = pd.read_csv("../data_clean/common_player_info_ready.csv")
df_combine = pd.read_csv("../data_clean/combine_ready.csv")

# Confirmar shapes
print("common_player_info_ready:", df_common.shape)
print("combine_ready:", df_combine.shape)


common_player_info_ready: (4171, 37)
combine_ready: (1199, 38)


In [8]:
# ======================================================
# 1. Crear conjuntos de jugadores
# ======================================================
set_player  = set(df_player.loc[df_player["player_name_clean"].notna(), "player_name_clean"])
set_common  = set(df_common.loc[df_common["player_name_clean"].notna(), "player_name_clean"])
set_combine = set(df_combine.loc[df_combine["player_name_clean"].notna(), "player_name_clean"])

# ======================================================
# 2. Calcular intersecciones
# ======================================================
inter_player_common  = set_player & set_common
inter_player_combine = set_player & set_combine
inter_all            = set_player & set_common & set_combine

# ======================================================
# 3. Resultados generales
# ======================================================
print("Exploración de jugadores:")
print(f"- Total en player_data: {len(set_player)}")
print(f"- Total en common_player_info_ready: {len(set_common)}")
print(f"- Total en combine_ready: {len(set_combine)}\n")

print(f"- Coinciden player ∩ common: {len(inter_player_common)}")
print(f"- Coinciden player ∩ combine: {len(inter_player_combine)}")
print(f"- Coinciden en las 3 tablas: {len(inter_all)}")

# ======================================================
# 4. Jugadores exclusivos de player_data
# ======================================================
solo_player = set_player - (set_common | set_combine)
print(f"- Solo en player_data: {len(solo_player)}")

# Mostrar algunos nombres como ejemplo
print("\nEjemplo de jugadores solo en player_data:")
for name in list(solo_player)[:20]:
    print("-", name)



Exploración de jugadores:
- Total en player_data: 4500
- Total en common_player_info_ready: 4139
- Total en combine_ready: 1176

- Coinciden player ∩ common: 3365
- Coinciden player ∩ combine: 549
- Coinciden en las 3 tablas: 470
- Solo en player_data: 1056

Ejemplo de jugadores solo en player_data:
- Chips Sobek
- Glen Combs
- Gene Williams
- Bob Feerick
- Lloyd Neal
- John Hazen
- Nazr Mohammed
- George Stone
- Red Morrison
- Willie Williams
- Chuck Gardner
- Dave Zeller
- Jerome Dyson
- Roy Ebron
- Fred Hilton
- Ime Udoka
- Jeffrey Congdon
- Fred Scolari
- Reggie Lacefield
- Ken Spain


## Resultados del merge exploratorio

- `player_data` tiene la base más amplia de jugadores (4500).
- `common_ready` aporta información más detallada, con 4139 jugadores.
- `combine_ready` es la más específica, con solo 1176 (principalmente de la era moderna del Draft Combine).
- Intersecciones:
  - `player_data ∩ common_ready`: 3365 jugadores.
  - `player_data ∩ combine_ready`: 549 jugadores.
  - En las 3 tablas: 470 jugadores (dataset núcleo con información completa).
- Exclusivos de `player_data`: 1056 jugadores, muchos históricos sin datos antropométricos o de combine.

Esto confirma que el **merge definitivo** debe hacerse con `outer join`, para no perder jugadores exclusivos de alguna tabla.


## Estrategia para composición corporal

- Se decidió calcular **BMI, %grasa estimada, masa grasa y masa magra directamente en `player_data`**.
- Justificación:
  - Es el dataset más amplio (4500 jugadores).
  - Garantiza que incluso los jugadores no presentes en `common_ready` o `combine_ready` tengan métricas antropométricas.
  - Reduce la proporción de nulos al hacer el merge final de jugadores.
- Con esto se construye una **base homogénea de composición corporal**, que luego se puede enriquecer con datos más precisos del Draft Combine cuando estén disponibles.


In [9]:
# ======================================================
# 1. Calcular BMI
# ======================================================
df_player["BMI"] = (df_player["weight_kg"] / (df_player["height_m"] ** 2)).round(1)

# ======================================================
# 2. Calcular % de grasa estimado (Deurenberg, usando age_at_last_season)
# Fórmula: 1.20 * BMI + 0.23 * edad - 16.2
# ======================================================
df_player["body_fat_pct_est"] = (
    1.20 * df_player["BMI"] + 0.23 * df_player["age_at_last_season"] - 16.2
).round(1)

# ======================================================
# 3. Masa grasa (kg)
# ======================================================
df_player["fat_mass_kg"] = (
    (df_player["body_fat_pct_est"] / 100) * df_player["weight_kg"]
).round(1)

# ======================================================
# 4. Masa magra (kg)
# ======================================================
df_player["lean_mass_kg"] = (
    df_player["weight_kg"] - df_player["fat_mass_kg"]
).round(1)

# ======================================================
# 5. Vista rápida de las nuevas métricas
# ======================================================
cols_check = [
    "player_name_clean", "height_m", "weight_kg", "BMI", "age_at_last_season",
    "body_fat_pct_est", "fat_mass_kg", "lean_mass_kg"
]

display(df_player[cols_check].head(15))


,player_name_clean,height_m,weight_kg,BMI,age_at_last_season,body_fat_pct_est,fat_mass_kg,lean_mass_kg
0,Alaa Abdelnaby,2.1,108.9,24.7,27,19.6,21.3,87.6
1,Zaid Abdulaziz,2.1,106.6,24.2,32,20.2,21.5,85.1
2,Kareem Abduljabbar,2.2,102.1,21.1,42,18.8,19.2,82.9
3,Mahmoud Abdulrauf,1.9,73.5,20.4,32,15.6,11.5,62.0
4,Tariq Abdulwahad,2.0,101.2,25.3,29,20.8,21.0,80.2
5,Shareef Abdurrahim,2.1,102.1,23.2,32,19.0,19.4,82.7
6,Tom Abernethy,2.0,99.8,25.0,27,20.0,20.0,79.8
7,Forest Able,1.9,81.6,22.6,25,16.7,13.6,68.0
8,John Abramovic,1.9,88.5,24.5,29,19.9,17.6,70.9
9,Alex Abrines,2.0,86.2,21.6,25,15.5,13.4,72.8


In [10]:
# ======================================================
# 1. Jugadores con composición corporal en cada tabla
# ======================================================

# Player_data → tiene %grasa estimado (body_fat_pct_est)
set_comp_player = set(
    df_player.loc[df_player["body_fat_pct_est"].notna(), "player_name_clean"]
)

# Common_ready → también tiene %grasa estimado
set_comp_common = set(
    df_common.loc[df_common["body_fat_pct_est"].notna(), "player_name_clean"]
)

# Combine_ready → tiene %grasa real (body_fat_pct)
set_comp_combine = set(
    df_combine.loc[df_combine["body_fat_pct"].notna(), "player_name_clean"]
)

# ======================================================
# 2. Resultados individuales
# ======================================================
print("Jugadores con composición corporal disponible:")
print(f"- En player_data: {len(set_comp_player)}")
print(f"- En common_ready: {len(set_comp_common)}")
print(f"- En combine_ready: {len(set_comp_combine)}\n")

# ======================================================
# 3. Intersecciones
# ======================================================
inter_player_common  = set_comp_player & set_comp_common
inter_player_combine = set_comp_player & set_comp_combine
inter_common_combine = set_comp_common & set_comp_combine
inter_all            = set_comp_player & set_comp_common & set_comp_combine

print("Coincidencias:")
print(f"- Player ∩ Common: {len(inter_player_common)}")
print(f"- Player ∩ Combine: {len(inter_player_combine)}")
print(f"- Common ∩ Combine: {len(inter_common_combine)}")
print(f"- En las 3 tablas: {len(inter_all)}\n")

# ======================================================
# 4. Cobertura global
# ======================================================
all_with_comp = set_comp_player | set_comp_common | set_comp_combine
print(f"- Total con composición corporal en al menos una tabla: {len(all_with_comp)}")



Jugadores con composición corporal disponible:
- En player_data: 4465
- En common_ready: 4043
- En combine_ready: 986

Coincidencias:
- Player ∩ Common: 3284
- Player ∩ Combine: 489
- Common ∩ Combine: 596
- En las 3 tablas: 409

- Total con composición corporal en al menos una tabla: 5534


### Definición de composición corporal en el análisis

Un jugador fue considerado con **datos de composición corporal** si en al menos una de las tablas (`player_data`, `common_ready`, `combine_ready`) presentaba alguna de estas métricas:

- Porcentaje de grasa corporal (%grasa).  
- Masa grasa en kilogramos.  
- Masa magra o libre de grasa en kilogramos.

Esto asegura que incluso si falta un cálculo completo, se aproveche cualquier indicador disponible.


## Cobertura de composición corporal

- Gracias al cálculo en `player_data`, se logró que casi todos los jugadores tengan estimación de composición corporal.
- `common_ready` complementa y valida datos, y `combine_ready` aporta mediciones físicas más detalladas.
- La cobertura global es de 5528 jugadores con algún dato de composición corporal.
- Existe un núcleo de 409 jugadores con datos consistentes en las tres fuentes, ideal para validaciones comparativas.


In [11]:
# ======================================================
# Cierre del análisis de player_data
# ======================================================

# Exploración inicial:
# - Se revisaron dimensiones, tipos de datos y valores nulos
# - Se detectaron inconsistencias en nombres, alturas y posiciones

# Limpieza aplicada:
# - Normalización de nombres de jugadores
# - Conversión de altura y peso a métricas internacionales
# - Cálculo de edad al debut y en la última temporada
# - Limpieza y estandarización de posiciones
# - Imputación de nulos en college
# - Eliminación de columnas redundantes

# Métricas derivadas:
# - Cálculo de BMI
# - % de grasa estimado (fórmula Deurenberg)
# - Masa grasa y masa magra en kg

# Finalidad:
# El dataset queda listo para integrarse con common_player_info_ready
# y combine_ready, permitiendo análisis conjuntos de rendimiento
# y composición corporal en jugadores de la NBA.

# Guardar dataset limpio
df_player.to_csv("../data_clean/player_data_ready.csv", index=False)

print("Dataset final guardado en ../data_clean/player_data_ready.csv")

# Vista rápida de verificación
print("\nPrimeras filas del dataset limpio:")
print(df_player.head())


Dataset final guardado en ../data_clean/player_data_ready.csv

Primeras filas del dataset limpio:
                  name  year_start  year_end birth_date  \
0       Alaa Abdelnaby        1991      1995 1968-06-24   
1      Zaid Abdul-Aziz        1969      1978 1946-04-07   
2  Kareem Abdul-Jabbar        1970      1989 1947-04-16   
3   Mahmoud Abdul-Rauf        1991      2001 1969-03-09   
4    Tariq Abdul-Wahad        1998      2003 1974-11-03   

                                 college   player_name_clean  height_m  \
0                        Duke University      Alaa Abdelnaby       2.1   
1                  Iowa State University      Zaid Abdulaziz       2.1   
2  University of California, Los Angeles  Kareem Abduljabbar       2.2   
3             Louisiana State University   Mahmoud Abdulrauf       1.9   
4              San Jose State University    Tariq Abdulwahad       2.0   

   weight_kg  age_at_debut  age_at_last_season position_all position_main  \
0      108.9            2